# Practical Week 11: Sequence-to-sequence models

There are three important tasks to complete this week:

1. Learning how to use LSTM and Transformers to process data sequences. You will find yourself waiting a long time
   for many of these computations to complete. While you wait:

2. Complete the course survey. It's in ilearn, and this link will show you how to view it and complete it:
https://ishare.mq.edu.au/prod/items/912f85f6-70e8-48a5-94ca-d3928e9d7c3c/1/viewcontent/


3. Sign up to get API access to OpenAI, which you will need for next week's practical. https://platform.openai.com/signup 



## Sequence to sequence

We're going to do a task that even ChatGPT-4 doesn't get right: arithmetic on long integer sequences.

The following cell creates a list of example bitwise-or calculations using python's `|` operator.

If you aren't familiar with Python's bit-handling functions, or what a bitwise-or does, here's a short
introduction: https://realpython.com/python-bitwise-operators/#bitwise-logical-operators

In [ ]:
math_facts = []
for i in range(64):
    for j in range(64):
        k = i | j
        math_facts.append(f"{i:06b} | {j:06b} = {k:06b}.")

Import the random library (or any other way you like) to see a few samples from the `math_facts` list.

### Data Preparation

We need to process these strings so that they can be used in a sequence-to-sequence model.

- For each string, split it up on the equals sign.
- On the left hand side of the equals sign, add a space between each digit so that each digit is its own token
- On the right hand side of the equals sign, generate 9 training samples. Encode 0 as `tf.convert_to_tensor([1,0,0])`, 1 as `tf.convert_to_tensor([0,1,0])` and "." as `tf.convert_to_tensor([0,0,1])`

e.g. suppose you were working with the statement:

```001000 | 010101 = 011101.```

Then you want to turn this into:

```
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 =', 'next_token': '0'}, 
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 = 0', 'next_token': '1'}, 
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 = 0 1', 'next_token': '1'},
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 = 0 1 1', 'next_token': '1'},
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 = 0 1 1 1', 'next_token': '0'},
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 = 0 1 1 1 0', 'next_token': '1'},
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 = 0 1 1 1 0 1', 'next_token': '.'},
```

In [ ]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf

r = re.compile('(\d)(\d)(\d)(\d)(\d)(\d)'
               ' \| '
               '(\d)(\d)(\d)(\d)(\d)(\d)'
               ' = '
               '(\d)(\d)(\d)(\d)(\d)(\d)\.'
              )
data = []
token_lookup = {'0': [1,0,0], '1': [0,1,0]}

for fact in math_facts:
    g = r.search(fact)
    lhs = f"{g[1]} {g[2]} {g[3]} {g[4]} {g[5]} {g[6]} | "
    lhs += f"{g[7]} {g[8]} {g[9]} {g[10]} {g[11]} {g[12]} ="
    
    data.append({'context': lhs, 'next_token': token_lookup[g[13]]} )
    lhs += f" {g[13]}"
    data.append({'context': lhs, 'next_token': token_lookup[g[14]]} )
    lhs += f" {g[14]}"
    data.append({'context': lhs, 'next_token': token_lookup[g[15]]} )
    lhs += f" {g[15]}"
    data.append({'context': lhs, 'next_token': token_lookup[g[16]]} )
    lhs += f" {g[16]}"
    data.append({'context': lhs, 'next_token': token_lookup[g[17]]} )
    lhs += f" {g[17]}"
    data.append({'context': lhs, 'next_token': token_lookup[g[18]]} )
    lhs += f" {g[18]}"
    #data.append({'context': lhs, 'next_token': tf.convert_to_tensor([0,0,1])} )
    data.append({'context': lhs, 'next_token': [0,0,1]} )

df = pd.DataFrame.from_records(data)

View a sample of the `df` dataframe to confirm it matches what you expect.

Split the `df` dataframe up into train, validation and test data.

Create a `keras.layers.TextVectorization` object with the following constructor parameters:

- split="whitespace"
- standardize=None
- output_sequence_length=26 (the longest possible sequence)

Adapt it to the training data (the `context` column) and vectorise the training, validation and test texts.

(Bonus challenge: this is one of the few times where you can actually give the vectorizer a vocabulary, and
you don't have to adapt from training data. Can you figure out how?)

The target values need to be converted into tensorflow Tensors. Unfortunately, `tf.convert_to_tensor` doesn't
handle pandas Series correctly. Use this work-around:

`train_y = tf.convert_to_tensor(list(train.next_token))`

(Create `train_y`, `validation_y` and `test_y`).

Create a model with:

- an input layer

- an embedding layer (a 4-dimensional embedding space will be sufficient)

- a bidirectional LSTM layer with 16 neurons in it

- an output layer of 3 dense neurons with a softmax activation

Compile it, and use a loss function of `categorical_crossentropy`

Fit the model using the training data, add validation data and early stopping. You'll find that it 
reaches 100% accuracy after about 100-120 epochs.

Surprisingly, it doesn't ever overfit. Make a graph of it if you want to (or just watch the numbers
fly past as it trains).

While you are waiting for this to complete, it would be a great time to **do the student evaluation survey now**.

## Student evaluation survey

Set this value to true when you have done your student evaluation survey.

In [ ]:
I_have_completed_my_student_evaluation_survey = False

## OpenAI API key

Set this value to true when you have signed up for your API key.

Remember that the API key can only be viewed once, so save it somewhere safe

In [ ]:
I_have_signed_up_for_openai = False

------------

## Using the sequence-to-sequence model

Check the test accuracy. It should be very good!

Create a function that takes a bitwise-or arithmetic problem, an optional list of already-returned tokens and returns the next token.

e.g. 
`get_next_token('1 0 0 1 0 0 | 0 0 0 0 0 1', '1 0 0')` should return "1".

You will need to:

- concatentate the first and second arguments (with an equals sign (=) between them

- vectorize that input string

- use the model to predict the probabilities of the three options for the next token (0,1 or .)

- find the option that has the highest probability

- return that as a token

Test it out with a sample problem.

Now create a function `get_full_answer` that calls `get_next_token` repeatedly with more context each time
until it gets a `.`. 

Try it out with a sample problem

# Transformers (Optional)

Depending on how fast your computer is, how long it takes to do the student survey and so on, you 
might or might not have a lot of time left over.

If you have run through all the above very quickly, go through the
https://www.tensorflow.org/text/tutorials/transformer

The tutorial is slightly out-of-date (as almost everything involving transformers is!)
because it re-implements components that already exist in the `keras_nlp` library, but it
is otherwise excellent.